### Importación de librerías

In [1]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.options import Options

In [ ]:
import logging # Para generar logs
import datetime
import sys

### Variables a modificar para adaptar el código

### Configuración de logs

### Lógica del Scrapping